In [ ]:
!pip install langchain
#!pip install chromadb
!pip install -U langchain-chroma
!pip install pypdf
!pip install pytest
!pip install ollama

In [1]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
import time
from sklearn.decomposition import PCA
import numpy as np
from langchain_chroma import Chroma

In [2]:
document_loader = PyPDFDirectoryLoader("documents")
documents = document_loader.load()
print(documents[1])

entry 27 in Xref table invalid but object found


page_content='Neuron, Vol. 36, 585–596, November 14, 2002, Copyright 2002 by Cell Press
The Unfolded Protein Response Modulates
Disease Severity in Pelizaeus-Merzbacher Disease
an X-linked recessive pediatric disorder characterized
by three common genetic forms of disease: coding re-gion or splice site mutations, duplications of the wild-Cherie M. Southwood,
1James Garbern,1,3
Wei Jiang,1and Alexander Gow1,2,3,4
1Center for Molecular Medicine and Genetics
type PLP1 gene, and null alleles. These mutations yield2Department of Pediatrics
a broad spectrum of disease phenotypes from severe,3Department of Neurology
connatal disease to mild forms characterized by pure Wayne State University School of Medicine
spastic paraparesis (reviewed in Garbern et al., 1999; Detroit, Michigan 48201
Southwood and Gow, 2001). Mutant alleles that model
all three of these genetic forms of PMD are available inmice, including (1) myelin synthesis-deficient (msd), an Summary
A242V missense mutation causing sev

In [3]:
# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1800,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)
# Split documents into chunks
chunks = text_splitter.split_documents(documents)

In [4]:
# Initialize embedding model
ollama_emb = OllamaEmbeddings(model="llama3")

In [5]:
# Initialize ChromaDB
CHROMA_PATH = "chroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

In [6]:
# Perform PCA on the embeddings before adding them to ChromaDB
#def apply_pca(embeddings, n_components=50):
#    pca = PCA(n_components=n_components)
#    reduced_embeddings = pca.fit_transform(embeddings)
#    return reduced_embeddings

last_page_id = None
current_chunk_index = 0
chunk_texts = []  # Store the text of chunks
chunk_metadata = []  # Store metadata to later add to ChromaDB

# Iterate over chunks to generate metadata
for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"

    # If the page ID is the same as the last one, increment the index.
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Calculate the chunk ID.
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id

    # Add the ID to the chunk metadata
    chunk.metadata["id"] = chunk_id
    chunk_metadata.append(chunk.metadata)
    chunk_texts.append(chunk.page_content)

In [7]:
# Define batch size
batch_size = 1000  # Adjust this value based on your performance needs

# Start time measurement
start_time = time.time()

# Function to split chunk_texts into batches
def create_batches(chunk_texts, batch_size):
    for i in range(0, len(chunk_texts), batch_size):
        yield chunk_texts[i:i + batch_size]

# Initialize list to store embeddings
all_embeddings = []

# Process and embed chunks in batches
for batch in create_batches(chunk_texts, batch_size):
    batch_embeddings = ollama_emb.embed_documents(batch)
    all_embeddings.extend(batch_embeddings)

# End time measurement
end_time = time.time()

# Calculate and print the time taken
time_taken = end_time - start_time
print(f"⏱️ Tiempo tomado para embeber los chunks en lotes: {time_taken:.2f} segundos")


⏱️ Tiempo tomado para embeber los chunks en lotes: 491.43 segundos


In [8]:
# Start time measurement
#start_time = time.time()

# Apply PCA to reduce the dimensionality of embeddings
embeddings = np.array(all_embeddings)
#reduced_embeddings = apply_pca(embeddings, n_components=50)

# End time measurement
#end_time = time.time()

# Calculate and print the time taken
#time_taken = end_time - start_time
#print(f"⏱️ Tiempo tomado para aplicar PCA a los embeddings: {time_taken:.2f} seconds")

In [9]:
# Start time measurement
start_time = time.time()

# Add or Update the documents in ChromaDB
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = [chunk for chunk in chunks if chunk.metadata["id"] not in existing_ids]
new_embeddings = [embeddings[i] for i, chunk in enumerate(chunks) if chunk.metadata["id"] not in existing_ids]

# Batch processing
BATCH_SIZE = 1000
try:
    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        for start in range(0, len(new_chunks), BATCH_SIZE):
            print(f"Adding chunk batch {start/BATCH_SIZE}")
            batch = new_chunks[start:start + BATCH_SIZE]
            batch_embeddings = new_embeddings[start:start + BATCH_SIZE]
            batch_ids = [chunk.metadata["id"] for chunk in batch]
            # Add the documents to the database with PCA-reduced embeddings
            db.add_documents(batch, embeddings=batch_embeddings, ids=batch_ids)
        print("✅ New documents added successfully")
    else:
        print("✅ No new documents to add")
except Exception as e:
    print(f"Error occurred during ingestion: {e}")

# End time measurement
end_time = time.time()

# Calculate and print the time taken
time_taken = end_time - start_time
print(f"⏱️ Tiempo tomado para ingestar los embeddings a ChromaDB: {time_taken:.2f} seconds")

Number of existing documents in DB: 0
👉 Adding new documents: 1896
Adding chunk batch 0.0
Adding chunk batch 1.0
✅ New documents added successfully
⏱️ Tiempo tomado para ingestar los embeddings a ChromaDB: 497.14 seconds


In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

#question="Can you give me some common issues from all diseases you know about?"
#question="Do you know how chromosome affects Fabry's disease?"
question="How can you relate Fabry disease with any other disease you know about?"

# Search the DB.
results = db.similarity_search_with_score(question, k=7)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)
# print(prompt)

model = Ollama(model="llama3")
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"

print(formatted_response)

Response: Based on the provided references, Fabry disease is a genetic disorder caused by a deficiency of alpha-galactosidase A enzyme. This deficiency leads to the accumulation of globotriaosylsphingosine (lyso-Gb3) in various tissues, causing symptoms such as pain, weakness, and organ damage.

One disease that can be related to Fabry disease is Gaucher disease, which is also caused by a deficiency of an enzyme involved in lipid metabolism. In both diseases, the accumulation of abnormal lipids leads to tissue damage and organ dysfunction. Both diseases are characterized by a build-up of sphingolipids, which can cause cellular toxicity and contribute to the development of symptoms.

Another example is Tay-Sachs disease, another lysosomal storage disorder caused by a deficiency of hexosaminidase A enzyme. Like Fabry disease, Tay-Sachs disease is characterized by the accumulation of gangliosides in neurons and other tissues, leading to progressive neurological degeneration and death.

Th